In [23]:
import pandas as pd

In [24]:
#Lectura del csv
df_temp = pd.read_csv('C:\\Users\\Federico\\Downloads\\Hackaton\\Project 1\\temperaturesbcndesde1780_2023_long.csv')
df_temp

#Temperaturas medias por mes y por año.
#238 rows × 13 columns

,Any,Mes,Desc_Mes,Temperatura
0,1780,1,Gener,6.7
1,1780,2,Febrer,7.3
2,1780,3,Març,11.6
3,1780,4,Abril,11.4
4,1780,5,Maig,16.3
...,...,...,...,...
2923,2023,8,Agost,26.0
2924,2023,9,Setembre,23.2
2925,2023,10,Octubre,20.2
2926,2023,11,Novembre,14.8


In [25]:
#Filtrado entre 1900 y 2023
df_filtro_1900_2023 = df_temp[(df_temp['Any'] >= 1900) & (df_temp['Any'] <= 2023)]

#Reseteo del índex
df_filtro_1900_2023.reset_index(drop=True, inplace=True)

#Eliminación de columna "Desc_Mes" -> No aportaba información relevante
df_filtro_1900_2023 = df_filtro_1900_2023[['Any', 'Mes', 'Temperatura']]

#Unificación en catalán para mantener un mismo idioma
df_filtro_1900_2023.rename(columns={'Temperatura': 'Temperature'}, inplace=True)

#Visualización del df final
df_filtro_1900_2023

,Any,Mes,Temperature
0,1900,1,8.0
1,1900,2,10.5
2,1900,3,8.8
3,1900,4,13.1
4,1900,5,16.3
...,...,...,...
1483,2023,8,26.0
1484,2023,9,23.2
1485,2023,10,20.2
1486,2023,11,14.8


In [27]:
#Agrupo por año y calculo el promedio de temperatura por cada año
df_avg_per_year = round(df_filtro_1900_2023.groupby('Any', as_index=False)['Temperature'].mean(), 0)

#Calculo el valor máximo y mínimo de las temperaturas promedio por año
max_temp_value = df_avg_per_year['Temperature'].max()
min_temp_value = df_avg_per_year['Temperature'].min()

#Calculo la distancia hasta el máximo y mínimo para cada año
df_avg_per_year['Distancia_max'] = abs(df_avg_per_year['Temperature'] - max_temp_value)
df_avg_per_year['Distancia_min'] = abs(df_avg_per_year['Temperature'] - min_temp_value)

#Calculo el índice de peligrosidad interpolado en la escala de 1 a 10: min_temp_value = 1 | max_temp_value = 10
df_avg_per_year['Index_vulnerabilitat'] = 1 + ((df_avg_per_year['Temperature'] - min_temp_value) / (max_temp_value - min_temp_value)) * (10 - 1)

#Creo un nuevo df con las columnas necesarias
df_filtro_1900_2023_abs = df_avg_per_year[['Any', 'Temperature','Index_vulnerabilitat']]

#Muestro el nuevo DataFrame
df_filtro_1900_2023_abs

,Any,Temperature,Index_vulnerabilitat
0,1900,15.0,4.6
1,1901,14.0,2.8
2,1902,14.0,2.8
3,1903,14.0,2.8
4,1904,15.0,4.6
...,...,...,...
119,2019,17.0,8.2
120,2020,17.0,8.2
121,2021,16.0,6.4
122,2022,18.0,10.0


In [28]:
#Exportación en formato .json para generar el endpoint ✅
df_filtro_1900_2023_abs.to_json('Indice_vulnerabilidad_temperature_1900-2023.json', orient='records', lines=True)